In [146]:
import torch

In [147]:
torch.cuda.is_available()

True

In [148]:
torch.cuda.get_device_name()

'Tesla T4'

In [149]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [150]:
# memory taken by GPU
torch.cuda.memory_allocated()

12140544

In [151]:
a = torch.Tensor([1, 2])
a.device

device(type='cpu')

In [152]:
b = torch.Tensor([3, 4]).cuda()
b.device

device(type='cuda', index=0)

In [153]:
# memory taken by GPU
torch.cuda.memory_allocated()

12141056

### Sending Models to GPU

In [154]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [155]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [156]:
class Model(nn.Module):
    def __init__(self, in_features=8, h1=16, h2=9, out_features=1):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.out(x))
        return x

In [157]:
torch.manual_seed(42)
model = Model()

In [158]:
next(model.parameters()).is_cuda

False

In [159]:
model.to(device)

Model(
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=1, bias=True)
)

In [160]:
## or you can also do
model = model.cuda()

In [161]:
next(model.parameters()).is_cuda

True

In [162]:
# reading custom dataset
df = pd.read_csv('sample_data/california_housing_train.csv')

In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           17000 non-null  float64
 1   latitude            17000 non-null  float64
 2   housing_median_age  17000 non-null  float64
 3   total_rooms         17000 non-null  float64
 4   total_bedrooms      17000 non-null  float64
 5   population          17000 non-null  float64
 6   households          17000 non-null  float64
 7   median_income       17000 non-null  float64
 8   median_house_value  17000 non-null  float64
dtypes: float64(9)
memory usage: 1.2 MB


In [164]:
X = df.drop('median_house_value', axis = 1).values
type(X)

numpy.ndarray

In [165]:
X.shape

(17000, 8)

In [166]:
y = df['median_house_value'].values

In [167]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3)

In [168]:
# send data to gpu
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.FloatTensor(y_train).cuda()
y_test = torch.FloatTensor(y_test).cuda()

In [169]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [171]:
import time
start = time.time()
epochs = 1000
losses = []

for i in range(epochs):

  y_pred = model.forward(X_train)
  loss = torch.sqrt(criterion(y_pred, y_train))
  losses.append(loss)

  # a neat trick to save screen space:
  if i%10 == 1:
      print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 169340.37500000
epoch: 11  loss: 168985.64062500
epoch: 21  loss: 168433.76562500
epoch: 31  loss: 167948.46875000
epoch: 41  loss: 167382.35937500
epoch: 51  loss: 166789.43750000
epoch: 61  loss: 166154.85937500
epoch: 71  loss: 165454.82812500
epoch: 81  loss: 164687.79687500
epoch: 91  loss: 163836.78125000
epoch: 101  loss: 162882.90625000
epoch: 111  loss: 161802.37500000
epoch: 121  loss: 160563.78125000
epoch: 131  loss: 159129.56250000
epoch: 141  loss: 157485.56250000
epoch: 151  loss: 155590.56250000
epoch: 161  loss: 153514.53125000
epoch: 171  loss: 151114.71875000
epoch: 181  loss: 148365.40625000
epoch: 191  loss: 145193.01562500
epoch: 201  loss: 141588.40625000
epoch: 211  loss: 137661.14062500
epoch: 221  loss: 133411.21875000
epoch: 231  loss: 129112.39843750
epoch: 241  loss: 125046.50000000
epoch: 251  loss: 121559.07031250
epoch: 261  loss: 119002.25000000
epoch: 271  loss: 117400.32812500
epoch: 281  loss: 116618.73437500
epoch: 291  loss: 116307